ABMs and LLMs, surveys:  
#### [Large language models empowered agent-based modeling and simulation: a survey and perspectives](https://www.nature.com/articles/s41599-024-03611-3) in BibDesk  
#### [Carbon and Silicon, Coexist or Compete? A Survey on Human-AI Interactions in Agent-based Modeling and Simulation](https://arxiv.org/pdf/2502.18145)  in BibDesk

Others:  
[Algorithmic Collusion by Large Language Models](https://arxiv.org/abs/2404.00806) in BibDesk  
[LLM-Driven Social Influence for Cooperative Behavior in Multi-Agent Systems](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=10912445) in BibDesk
  
Technicalities on [Multi-Agent System Powered by Large Language Models: An Innovation Guide](https://medium.com/data-science-collective/multi-agent-system-powered-by-large-language-models-an-innovation-guide-e5cc9dd6f366)

Walrasian auctioneer or crier?  
Not an auction market, simply a market where someone–-the crier–-proposes prices, following a trial and error process, to search for an equlibrium. See https://en.wikipedia.org/wiki/Walrasian_auction

useful paper https://air.unimi.it/bitstream/2434/43747/1/walras_lombardo_07fin.pdf (a copy in this folder)

A translation error, look at *Walrasian auction* https://en.wikipedia.org/wiki/Walrasian_auction

Using **[Swarm (experimental, educational)](https://github.com/openai/swarm/blob/main/README.md)** con **[Orchestrating Agents: Routines and Handoffs](https://cookbook.openai.com/examples/orchestrating_agents)**

other tools: [LangGraph vs CrewAI vs OpenAI Swarm: Which AI Agent Framework to Choose?](https://oyelabs.com/langgraph-vs-crewai-vs-openai-swarm-ai-agent-framework/)

If missing  
! pip install git+https://github.com/openai/swarm.git  

In [1]:
from swarm import Swarm, Agent
from dotenv import load_dotenv
import tool
import os

In [ ]:
load_dotenv()
os.getenv('OPENAI_API_KEY');

In [2]:
client = Swarm()

In [ ]:
#client.client.api_key

In [ ]:
client.client.api_key = tool.tool()

In [3]:
import random as r

def setRandomPrice() -> float:
    """generate a random price"""
    return 100 * r.random()

## Introducing the crier, suggesting how to operate, asking very concise replies; the price level is set randomly

In [11]:
#for the buyer-seller
def instructions(context_variables):
   price_level = context_variables["price_level"]
   rules = "1. Use the function setRandomPrice " +\
           "to set your reservation price as a random price and report "+\
           "it as a pure number without type of currency specification. "+\
           "2. Report your id using the sequence buyerSeller"+str(i)+". "+\
           "3. Report if you buy or sell comparing you reservation price and the price level, "+\
           "considering that if your reservation price is greater that the price level is convenient to buy "+\
           "and that in the opposite case is convenient to sell. "+\
           "If your reservation price is very close to the price level, do nothing "+\
           f"The price level is {price_level} "+\
            "finally, explain your reasoning"
   return rules

In [12]:
lA=[]
for i in range(10):
 lA.append(
    Agent(
    model="gpt-4o",
    name="Agent buying-selling",
    instructions = instructions,
    functions= [setRandomPrice]
    )
 )

In [7]:
#for the crier
def instructions2(context_variables):
    global pl
    pl=100 * r.random()
    print("price level ",pl)
    rules = "Report your id as 'crier'"
    return rules  

In [8]:
crier = Agent(
    model="gpt-4o",
    name="Agent crier",
    instructions = instructions2
    )

In [14]:
messages = [{"role": "user", "content": "Report your id"}]
response = client.run( agent=crier, 
                       messages=messages)
print(response.messages[-1]["content"])

price level  57.38711951661873
My id is 'crier'.


In [16]:
for i in range(10):
 messages = [{"role": "user", "content": "Report your id, what is your reservation price and what do you do;"+
                                         "make your report in a very concise way"}]
 response = client.run(
                       agent=lA[i], 
                       messages=messages,
                       context_variables={"price_level": pl})
 print(response.messages[-1]["content"])

ID: buyerSeller0  
Reservation Price: 66.71  
Action: Buy  

Reasoning: My reservation price (66.71) is greater than the price level (57.39), indicating it's advantageous to buy.
ID: buyerSeller1  
Reservation Price: 8.417048978199048  
Action: Sell  

Reasoning: My reservation price (8.417048978199048) is significantly lower than the price level (57.38711951661873), indicating it is advantageous to sell.
ID: buyerSeller2, Reservation Price: 2.2451385756530273. Given my reservation price is less than the price level (57.38711951661873), I will sell.
ID: buyerSeller3, Reservation Price: 0.6711454470165745, Action: Sell. 

Reasoning: My reservation price (0.6711454470165745) is significantly lower than the price level (57.38711951661873), making it advantageous to sell.
ID: buyerSeller4, Reservation Price: 93.49325509022819, Action: Buy (Reservation price is greater than the price level 57.38711951661873)
ID: buyerSeller5, Reservation Price: 10.559614528433169, Action: Sell (since reserv